In [ ]:

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [33]:
import os
import time
from datetime import date
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException


In [60]:
from bs4 import BeautifulSoup
import requests

# url="https://www.amazon.in/Ainclu-Womens-Classic-Layered-Lace-Up/dp/B00K69XHGY/ref=b2b_gw_d_csf_b2b_sccl_1/258-8837396-8497212?pd_rd_w=SvU1u&content-id=amzn1.sym.80118820-a13a-4810-a5d3-0816775952fa&pf_rd_p=80118820-a13a-4810-a5d3-0816775952fa&pf_rd_r=80CBD46BM18R81225Y26&pd_rd_wg=0hL82&pd_rd_r=c516329a-4a79-48da-8432-a3ec36ed9d0f&pd_rd_i=B083QDVK83&psc=1"
# result=requests.get(url)
# doc=BeautifulSoup(result.text,"html.parser")
# prices=doc.find_all(text="₹")
# parent=prices[0].parent
# print(parent)

# #tags=doc.find_all(["p","div",";li"])

# #tags=doc.find_all(text=re.compile("\$"),limit=1)
# #for tag in tags:
#    #print(tag.strip())
from requests_html import HTMLSession
import json
import time
#Python is an “object-oriented programming language.” This means that almost all the code is 
#implemented using a special construct called classes. Programmers use classes to keep related 
#things together. This is done using the keyword “class,” which is a grouping of object-oriented 
#constructs.
#A class is a code template for creating objects. 
#Objects have member variables and have behaviour associated with them.
#An object is created using the constructor of the class. 
#This object will then be called the instance of the class. In Python we create instances in the following manner
#Instance = class(arguments)
class Reviews:
    def __init__(self) -> None:
        self.session=HTMLSession() #this is our object tht we ll use whenever class instance is called
        self.headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36', 'Origin':'https://www.flipkart.com', 'Referer': 'https://www.flipkart.com/'}
        #self.url= f'https://www.amazon.in/Rovars-U2301-Boys-Black-Light/product-reviews/{self.asin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
        self.url=None
        #self.driver = None
        self.product_name = None
        self.formatted_product_name = None
        self.product_url=None
        self.product_page_url=None
       
        # to get to amazon india website
    # def open_amazon_website(self):
    #     opt=Options()# creates instance for options
    #     opt.add_argument("--disable-infobars")
    #     opt.add_argument("--disable-extensions")
    #     opt.add_argument('--log-level=OFF')
    #     opt.add_experimental_option('excludeSwitches', ['enable-logging'])
    #     url = "https://www.amazon.in/"
    #     self.driver = webdriver.Chrome(ChromeDriverManager(driver_version='114.0.5735.90').install())
    #     # go to that page using driver
    #     self.driver.get(url)
    #     #to load the website
    #     time.sleep(3)
     #https://www.flipkart.com/search?q=PS5+Digital+Standalone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&as-pos=1&as-type=HISTORY   
            
        #creating url to get to product page eg mobile phones,playstation
    def open_all_product_page(self):
        s=input("\n>>Enter the product name")
        self.product_name=s
        self.formatted_product_name=self.product_name.replace(" ","+")
        
        #fits product name in url
        self.product_url = "https://www.flipkart.com/search?q={}&otracker"
        self.product_url = self.product_url.format(self.formatted_product_name)
        
        print(">> Product URL: ", self.product_url)
        

            
        #to get links of all products in the page
    
    def get_product_link(self):
        # product_name=input('\n>>Enter product name')
        # self.product_name=product_name
        # self.formatted_product_name=self.product_name.replace(" ","-")
        result=requests.get(self.product_url, headers=self.headers)
        soup=BeautifulSoup(result.content,"html.parser")
       
        #links=soup.find_all("a",attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" })
        #links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
        # Get all elements which have an attribute data-id
        elems = soup.find_all(attrs={'data-id': True})
        firstelement=elems[0]
        # Get all links from the first element
        childelement = firstelement.find_all('a')
        # Get the first child link
        firstchild = childelement[0]        
        self.product_page_url= 'https://www.flipkart.com' + firstchild['href']
       
        print(">>Product page URL:",self.product_page_url)    
                
    def get_review_url(self):
        self.url=self.product_page_url.replace("/p/","/product-reviews/")
        print(">>Review page URL:",self.url)
        
    #all reviews from all review pages of one product
    def find_all_reviews(self):
        resultrev=requests.get(self.url,header=self.headers)
        #r=self.session.get(self.url)
        # if not r.html.find('div[data-hook=review]'):
        #     return False
        # else:
        #     return r.html.find('div[data-hook=review]')
        soup2=BeautifulSoup(resultrev.content,"html.parser")
        
        #<div class="t-ZTKy _1QgsS5"><div><div class="_3LWZlK _1BLPMq _3B8WaH">4<img src="data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHdpZHRoPSIxMyIgaGVpZ2h0PSIxMiI+PHBhdGggZmlsbD0iI0ZGRiIgZD0iTTYuNSA5LjQzOWwtMy42NzQgMi4yMy45NC00LjI2LTMuMjEtMi44ODMgNC4yNTQtLjQwNEw2LjUuMTEybDEuNjkgNC4wMSA0LjI1NC40MDQtMy4yMSAyLjg4Mi45NCA0LjI2eiIvPjwvc3ZnPg==" class="_1wB99o"></div><div class="_6K-7Co">Good</div><span class="_1H-bmy"><span>READ MORE</span></span></div></div>
        #<div class="_3LWZlK _32lA32 _1BLPMq _3B8WaH">2<img src="data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHdpZHRoPSIxMyIgaGVpZ2h0PSIxMiI+PHBhdGggZmlsbD0iI0ZGRiIgZD0iTTYuNSA5LjQzOWwtMy42NzQgMi4yMy45NC00LjI2LTMuMjEtMi44ODMgNC4yNTQtLjQwNEw2LjUuMTEybDEuNjkgNC4wMSA0LjI1NC40MDQtMy4yMSAyLjg4Mi45NCA0LjI2eiIvPjwvc3ZnPg==" class="_1wB99o"></div>
        all_reviews=soup2.find_all('div',attrs={'class':'t-ZTKy _1QgsS5'})
        all_ratings=all_reviews.div
       # <div class="_6K-7Co">This product is  very heavy it's not a comfortable shoes</div>
        all_body=soup2.find_all(attrs={'class':'_6K-7Co'}).text
        
        return all_ratings,all_body
        
        
          

   # review parsing
    def parse(self,all_ratings,all_body):
        total=[]  
        rating_count=0   
        ratings=0  
        avg_rating=0 
        total_sentimental_score=0
        for i in range(1,len(all_ratings)):
            #title=review.find('a[data-hook=review-title]',first=True).text
            rating=all_ratings[i]
            body=all_body[i].replace('\n','').strip()
            rating_count=rating_count+1
            ratings=ratings+(int(rating[0]))
            sentiment= sentiment_pipeline(body)[0]
            if sentiment['label']=='POSITIVE':
               total_sentimental_score=sentiment['score']+total_sentimental_score
            else:
               total_sentimental_score=total_sentimental_score-sentiment['score']
            
            data={
                'rating_count':rating_count,
               
                'rating':rating,
                'body':body[:100],
                'sentiment':sentiment
            }
            total.append(data)
        avg_rating=ratings/rating_count
        avg_sentimental_score=total_sentimental_score/rating_count
        return total,avg_rating,avg_sentimental_score
    
    
    
    def save(self,results):
        with open(self.product_name+'reviews.json','w') as f:
            json.dump(results,f)
#https://www.amazon.in/Sony-PS5-PlayStation-Console/dp/B0BRCP72X8
            
            
            
amz=Reviews()

amz.open_all_product_page()

amz.get_product_link()
amz.get_review_url()
results=[]
average_rating=0

    
time.sleep(0.3)
ratings,body=amz.find_all_reviews()
results.append(amz.parse(ratings,body))
temp,average_rating,avg_sentimental_score=amz.parse(ratings,body)
    

print(f"The average rating is: {average_rating}\n")
print(f"The average sentimental score is: {avg_sentimental_score}")
if avg_sentimental_score>0:
    print("Overall sentimental review is POSITIVE")
else:
    print("Overall sentimental review is NEGATIVE")

amz.save(results)         



>> Product URL:  https://www.flipkart.com/search?q=BRUTON+Trendy+Running+Shoes+Running+Shoes+For+Men&otracker
>>Product page URL: https://www.flipkart.com/aadi-trendy-lightweight-sports-walking-running-shoes-men/p/itmff8ab6884fbd9?pid=SHOGTYTAAGU6ZMMR&lid=LSTSHOGTYTAAGU6ZMMRWP1YXB&marketplace=FLIPKART&q=BRUTON+Trendy+Running+Shoes+Running+Shoes+For+Men&store=osp%2Fcil%2F1cu&srno=s_1_1&otracker=search&fm=organic&iid=en_0Z3gNrpiqd4iZonG71xt2MkYU-mQeKuV6wnlpr8vGShIbnKjOdfCGwsGFtp0899-Jiu3Vdp_bV9NtNWEglT2Jg%3D%3D&ppt=None&ppn=None&ssid=o4ax40nyzk0000001703874580061&qH=0405bdbc28919618
>>Review page URL: https://www.flipkart.com/aadi-trendy-lightweight-sports-walking-running-shoes-men/product-reviews/itmff8ab6884fbd9?pid=SHOGTYTAAGU6ZMMR&lid=LSTSHOGTYTAAGU6ZMMRWP1YXB&marketplace=FLIPKART&q=BRUTON+Trendy+Running+Shoes+Running+Shoes+For+Men&store=osp%2Fcil%2F1cu&srno=s_1_1&otracker=search&fm=organic&iid=en_0Z3gNrpiqd4iZonG71xt2MkYU-mQeKuV6wnlpr8vGShIbnKjOdfCGwsGFtp0899-Jiu3Vdp_bV9NtNWEglT2Jg%

TypeError: Session.request() got an unexpected keyword argument 'header'

>> Product URL:  https://www.flipkart.com/search?q=RUN+SEVEN+(+by+GO21+)+Soft+Insole,+Slip-Resistance|Walking+Casual+Sneakers+For+Men+Reviews&otracker
>>Product page URL: https://www.flipkart.com/run-seven-go21-soft-insole-slip-resistance-walking-casual-sneakers-men/p/itm5677c9cefe53f?pid=SHOGQHZZS5XKFF6E&lid=LSTSHOGQHZZS5XKFF6E6KYUIQ&marketplace=FLIPKART&q=RUN+SEVEN+%28+by+GO21+%29+Soft+Insole%2C+Slip-Resistance%7CWalking+Casual+Sneakers+For+Men+Reviews&store=search.flipkart.com&srno=s_1_1&otracker=search&fm=organic&iid=c231c79e-9bf6-4eda-83ee-851e520f4bea.SHOGQHZZS5XKFF6E.SEARCH&ppt=None&ppn=None&ssid=091374jpeo0000001703870864979&qH=706bc136d4c85cd3
No more pages
The average rating is: 0



NameError: name 'avg_sentimental_score' is not defined